In [3]:
import json
import plotly.graph_objects as go

def plot_sembas_results(file, num):
    """
    Simple 3D scatter plot using Plotly - automatically loads results.json
    """
    # Load data from JSON file
    with open(file, 'r') as f:
        data = json.load(f)
    
    coords = data[num]["requests"]
    results = data[num]["results"]
    
    # Extract coordinates
    x = [pt[0] for pt in coords]
    y = [pt[1] for pt in coords]
    z = [pt[2] for pt in coords]
    
    # Create color mapping (0 = red/failed, 1 = blue/successful)
    colors = [1 if res else 0 for res in results]
    
    # Create 3D scatter plot
    fig = go.Figure(data=[go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(
            size=10,
            color=colors,
            colorscale=[[0, 'red'], [1, 'blue']],
            opacity=1.0,
            line=dict(width=2, color='black')
        ),
        text=[f'Point {i+1}<br>{"Successful" if res else "Failed"}<br>({x[i]:.2f}, {y[i]:.2f}, {z[i]:.2f})'
              for i, res in enumerate(results)],
        hoverinfo='text'
    )])
    
    # Update layout
    fig.update_layout(
        title='Sembas Results - 3D Visualization',
        scene=dict(
            xaxis_title='Progress Weight',
            yaxis_title='Bounds Weight',
            zaxis_title='Proximity Weight'
        ),
        width=1000,
        height=800
    )
    
    fig.show()
    # fig.write_image("images/"+str(num)+'.png')


def plot_sembas_2d(file, dim1=0, dim2=1):
    """
    2D scatter plot using Plotly - extract 2 dimensions of your choosing
    
    Parameters:
    file: JSON file path
    dim1: First dimension index (0=Progress Weight, 1=Bounds Weight, 2=Proximity Weight)
    dim2: Second dimension index (0=Progress Weight, 1=Bounds Weight, 2=Proximity Weight)
    """
    # Load data from JSON file
    with open(file, 'r') as f:
        data = json.load(f)
    
    coords = data[0]["requests"]
    results = data[0]["results"]
    
    # Dimension labels
    dim_labels = ['Progress Weight', 'Bounds Weight', 'Proximity Weight']
    
    # Validate dimension indices
    if dim1 not in [0, 1, 2] or dim2 not in [0, 1, 2]:
        raise ValueError("Dimension indices must be 0, 1, or 2")
    
    if dim1 == dim2:
        raise ValueError("Dimension indices must be different")
    
    # Extract chosen dimensions
    x = [pt[dim1] for pt in coords]
    y = [pt[dim2] for pt in coords]
    
    # Create color mapping (0 = red/failed, 1 = blue/successful)
    colors = [1 if res else 0 for res in results]
    
    # Create 2D scatter plot
    fig = go.Figure(data=[go.Scatter(
        x=x,
        y=y,
        mode='markers',
        marker=dict(
            size=12,
            color=colors,
            colorscale=[[0, 'red'], [1, 'blue']],
            opacity=1.0,
            line=dict(width=2, color='black')
        ),
        text=[f'Point {i+1}<br>{"Successful" if res else "Failed"}<br>{dim_labels[dim1]}: {x[i]:.2f}<br>{dim_labels[dim2]}: {y[i]:.2f}'
              for i, res in enumerate(results)],
        hoverinfo='text'
    )])
    
    # Update layout
    fig.update_layout(
        title=f'Sembas Results - 2D Visualization ({dim_labels[dim1]} vs {dim_labels[dim2]})',
        xaxis_title=dim_labels[dim1],
        yaxis_title=dim_labels[dim2],
        width=800,
        height=600,
        showlegend=False
    )
    
    fig.show()

if __name__ == "__main__":
    # Example usage:

    # file = 'scalar_full_collision_test.json' # Volume: 0.1480389491275113, points: 667
    # plot_sembas_results(file)  # 3D plot
    
    # # 2D plots - choose any two dimensions:
    # plot_sembas_2d(file, dim1=0, dim2=1)  # Progress vs Bounds
    # plot_sembas_2d(file, dim1=0, dim2=2)  # Progress vs Proximity
    # plot_sembas_2d(file, dim1=1, dim2=2)  # Bounds vs Proximity
    pass

In [7]:
file = 'param_tuning.json' 

for i in range(5):
    plot_sembas_results(file, i)  # 3D plot


# 2D plots - choose any two dimensions:y
# plot_sembas_2d(file, dim1=0, dim2=1)  # Progress vs Bounds
# plot_sembas_2d(file, dim1=0, dim2=2)  # Progress vs Proximity
# plot_sembas_2d(file, dim1=1, dim2=2)  # Bounds vs Proximity

IndexError: list index out of range